In [1]:
import re
import numpy as np
import pandas as pd
from pprint import pprint
import nltk
from nltk.corpus import stopwords
import os.path

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
#from gensim.models.wrappers import LdaMallet


# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
%matplotlib inline


# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [2]:
data = pd.read_csv(r'C:\Users\gabov\Desktop\aerolinea_sentiment\data\data_final.csv')

In [3]:
# Convert to list
data = data.descripcion.values.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

['Hemos recibido su reporte srita Labrada nos disculpamos por los '
 'inconvenientes aspi mismo si hubo una confusión con la información que se '
 'brindo respecto a las políticas de su vuelo hemos validado que a su '
 'reservación SHZKR fue aplicada su reprogramación el día de mayo asi mismo '
 'anexamos la información correspondiente a su certificado electrónico por '
 'servicio de maleta adicional así como sus políticas de uso Número de '
 'certificado electrónico Titular Maura Labrada Araujo Monto MXN Vigencia OCT '
 'Es válido únicamente a través del call center Tiene una vigencia de días '
 'para hacerlo valido la fecha del vuelo puede ser posterior a esos días En '
 'caso de que el costo de su reservación sea mayor al monto de su voucher se '
 'tendrá que pagar el excedente No es residual En la reservación generada con '
 'él sólo puede ir un pasajero el titular del voucher Además el titular del '
 'voucher debe ser quién genere esta reservación Quedamos a sus órdenes para '
 'c

In [4]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['hemos', 'recibido', 'su', 'reporte', 'srita', 'labrada', 'nos', 'disculpamos', 'por', 'los', 'inconvenientes', 'aspi', 'mismo', 'si', 'hubo', 'una', 'confusion', 'con', 'la', 'informacion', 'que', 'se', 'brindo', 'respecto', 'las', 'politicas', 'de', 'su', 'vuelo', 'hemos', 'validado', 'que', 'su', 'reservacion', 'shzkr', 'fue', 'aplicada', 'su', 'reprogramacion', 'el', 'dia', 'de', 'mayo', 'asi', 'mismo', 'anexamos', 'la', 'informacion', 'correspondiente', 'su', 'certificado', 'electronico', 'por', 'servicio', 'de', 'maleta', 'adicional', 'asi', 'como', 'sus', 'politicas', 'de', 'uso', 'numero', 'de', 'certificado', 'electronico', 'titular', 'maura', 'labrada', 'araujo', 'monto', 'mxn', 'vigencia', 'oct', 'es', 'valido', 'unicamente', 'traves', 'del', 'call', 'center', 'tiene', 'una', 'vigencia', 'de', 'dias', 'para', 'hacerlo', 'valido', 'la', 'fecha', 'del', 'vuelo', 'puede', 'ser', 'posterior', 'esos', 'dias', 'en', 'caso', 'de', 'que', 'el', 'costo', 'de', 'su', 'reservacion', 

In [5]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['hemos_recibido', 'su', 'reporte', 'srita', 'labrada', 'nos', 'disculpamos', 'por', 'los', 'inconvenientes', 'aspi', 'mismo', 'si', 'hubo', 'una', 'confusion', 'con', 'la', 'informacion', 'que', 'se', 'brindo', 'respecto', 'las', 'politicas', 'de', 'su', 'vuelo', 'hemos', 'validado', 'que', 'su', 'reservacion', 'shzkr', 'fue', 'aplicada', 'su', 'reprogramacion', 'el', 'dia', 'de', 'mayo', 'asi', 'mismo', 'anexamos', 'la', 'informacion', 'correspondiente', 'su', 'certificado', 'electronico', 'por', 'servicio', 'de', 'maleta', 'adicional', 'asi', 'como', 'sus', 'politicas', 'de', 'uso', 'numero', 'de', 'certificado', 'electronico', 'titular', 'maura', 'labrada', 'araujo', 'monto', 'mxn', 'vigencia', 'oct', 'es', 'valido', 'unicamente', 'traves', 'del', 'call_center', 'tiene', 'una', 'vigencia', 'de', 'dias', 'para', 'hacerlo_valido', 'la', 'fecha', 'del', 'vuelo', 'puede', 'ser', 'posterior', 'esos', 'dias', 'en', 'caso', 'de', 'que', 'el', 'costo', 'de', 'su', 'reservacion', 'sea', 'ma

In [6]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    # return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
    return [[word for word in simple_preprocess(str(doc)) if word not in final_stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]


In [7]:
#Define stop words
stop_words_sp = set(stopwords.words('spanish'))
# stop_words_en = set(stopwords.words('english'))
stop_words = stop_words_sp #| stop_words_en
#add words that aren't in the NLTK stopwords list
new_stopwords = ["buenas", "buen","blueaerolinea","saludos cordiales","com mx","www","gmail com","mailto","gmail", "com","hotmail","image004","hola","tarde","gmail com","muchas","gracias", "buenos","día","cid","mx","ustede","correo", "electronico",
                          "saludos","cordiales", "formato","pdf","solo","ustedes","quedo","espera","noche", "blue", "aerobu","adjunto podrá", "términos", "condiciones","correo electrónico","saber", "si","podrás encontrar","quedo","espera","noche",
                          "adjunto","podrá", "png","podrás","encontrar","grupos","quedo","favor","hoy", "quisiera", "tardes","adjunto podrá", "términos condiciones","correo electrónico","saber", "si","image","6c756af0","noches","01d57aa3",
                      "com", "nbsp","cognitoforms","dia","quedo","gracias","vea","saludos","favor", "días","hola","image004","xx ","tarde","gmail com","muchas","gracias", "buenos","día","cid","mx","ustede","puedo", "hacer"]
new_stopwords_list = stop_words.union(new_stopwords)


not_stopwords = {} 
final_stop_words = set([word for word in new_stopwords_list if word not in not_stopwords])
print(final_stop_words)

{'ni', 'estábamos', 'seríais', 'cordiales', 'esto', 'solo', 'teníais', 'tuve', 'le', 'tuya', 'tuviste', 'es', 'estaremos', 'tuvieron', 'son', 'erais', 'dia', 'habíais', 'www', 'tendréis', 'todos', 'fui', 'tengas', 'estada', 'fueron', 'antes', 'mis', 'estuvisteis', 'tarde', 'tenido', 'algunas', 'nuestras', 'como', 'hayáis', 'podrás encontrar', 'seréis', 'esos', 'que', 'com', 'donde', 'cognitoforms', 'mailto', 'hotmail', 'tendrías', 'sois', 'nuestra', 'podrá', 'términos', 'formato', 'tuviésemos', 'fueseis', 'tendríais', 'sentid', 'estado', 'me', 'buenos', 'estemos', 'estar', 'de', 'fuisteis', 'mx', 'eran', 'estadas', 'se', 'fuimos', 'vuestras', 'vuestro', 'suya', 'sin', 'sentidos', 'fuiste', 'estaréis', 'ellas', 'esté', 'tuviesen', 'tened', 'estés', 'correo', 'poco', 'tuvieras', 'muy', 'les', 'quisiera', 'qué', 'sentido', 'png', 'sobre', 'tuyos', 'ustede', 'estás', 'hoy', 'teníamos', 'tendrían', 'tenía', 'sea', 'habrías', 'han', 'habiendo', 'grupos', 'con', 'sentida', 'eso', 'un', 'esa',

In [8]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

In [9]:
# Create Dictionary
# id2word = corpora.Dictionary(data_lemmatized)
id2word = corpora.Dictionary(data_words_bigrams)

# Create Corpus
# texts = data_lemmatized
texts = data_words_bigrams

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 2), (7, 1), (8, 1), (9, 1), (10, 1), (11, 2), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 2), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 3), (27, 1), (28, 2), (29, 1), (30, 1), (31, 1), (32, 1), (33, 2), (34, 2), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 2), (42, 2), (43, 2), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 4), (50, 1), (51, 1), (52, 2), (53, 2), (54, 1), (55, 1), (56, 1), (57, 3), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 2), (64, 3), (65, 2)]]


In [10]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('ademas', 1),
  ('adicional', 1),
  ('aleyda_garza', 1),
  ('anexamos', 1),
  ('aplicada', 1),
  ('araujo', 1),
  ('asi', 2),
  ('aspi', 1),
  ('brindo', 1),
  ('call_center', 1),
  ('caso', 1),
  ('certificado', 2),
  ('clientes', 1),
  ('confusion', 1),
  ('correspondiente', 1),
  ('costo', 1),
  ('cualquier', 1),
  ('debe', 1),
  ('dias', 2),
  ('disculpamos', 1),
  ('excedente', 1),
  ('fecha', 1),
  ('generada', 1),
  ('genere', 1),
  ('hacerlo_valido', 1),
  ('inconvenientes', 1),
  ('informacion', 3),
  ('ir', 1),
  ('labrada', 2),
  ('maleta', 1),
  ('maura', 1),
  ('mayo', 1),
  ('mayor', 1),
  ('mismo', 2),
  ('monto', 2),
  ('mxn', 1),
  ('numero', 1),
  ('oct', 1),
  ('ordenes', 1),
  ('pagar', 1),
  ('pasajero', 1),
  ('politicas', 2),
  ('posterior', 2),
  ('puede', 2),
  ('quedamos', 1),
  ('recibido', 1),
  ('reporte', 1),
  ('reprogramacion', 1),
  ('requiera', 1),
  ('reservacion', 4),
  ('residual', 1),
  ('respecto', 1),
  ('ser', 2),
  ('servicio', 2),
  ('shzkr

In [11]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=4, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [12]:
# Print the Keyword in the 4 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.027*"covid" + 0.021*"viajar" + 0.020*"viaje" + 0.018*"prueba" + '
  '0.015*"atencion" + 0.014*"numero" + 0.014*"boleto" + 0.013*"necesito" + '
  '0.013*"boletos" + 0.012*"dias"'),
 (1,
  '0.022*"reservacion" + 0.021*"cotizacion" + 0.016*"nombre" + 0.014*"clave" + '
  '0.010*"fecha" + 0.010*"solicito" + 0.009*"pago" + 0.008*"presente" + '
  '0.008*"respuesta" + 0.008*"apoyo"'),
 (2,
  '0.090*"vuelo" + 0.041*"julio" + 0.029*"mexico" + 0.023*"monterrey" + '
  '0.020*"cancun" + 0.020*"ciudad" + 0.018*"pm" + 0.017*"vuelos" + '
  '0.017*"regreso" + 0.013*"mty"'),
 (3,
  '0.016*"maleta" + 0.015*"mas" + 0.011*"vb" + 0.010*"aerolinea" + '
  '0.009*"vuelo" + 0.009*"equipaje" + 0.009*"aeropuerto" + 0.007*"manana" + '
  '0.007*"quiero" + 0.007*"informacion"')]


In [13]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_words_bigrams, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -8.574979549521306

Coherence Score:  0.6029944051045057


In [14]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.217846  0.110479       1        1  37.917355
3     -0.143392 -0.299818       2        1  25.627848
2      0.339478 -0.072530       3        1  19.957018
0      0.021760  0.261869       4        1  16.497778, topic_info=             Term          Freq         Total Category  logprob  loglift
65          vuelo  28348.000000  28348.000000  Default  30.0000  30.0000
655         julio  11424.000000  11424.000000  Default  29.0000  29.0000
178        mexico   8110.000000   8110.000000  Default  28.0000  28.0000
49    reservacion  11492.000000  11492.000000  Default  27.0000  27.0000
949         covid   6178.000000   6178.000000  Default  26.0000  26.0000
...           ...           ...           ...      ...      ...      ...
194        tiempo   1567.479081   1922.392224   Topic4  -4.9870   1.5978
247         poder   2113.640592   3676.005139   Topic4  -4.6880   1.2485
18           dias   2743.151052   7533.090626   Topic4  -4.4273   0.7917
1219        tenia   1513.485591   2716.385047   Topic4  -5.0220   1.2171
185     respuesta   1909.256078   7661.496156   Topic4  -4.7897   0.4125

[198 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
1212      4  0.999875     abordar
81        1  0.059323     aerobus
81        3  0.940396     aerobus
120       2  0.999835   aerolinea
768       2  0.999903  aeropuerto
...     ...       ...         ...
197       4  0.714876       viaje
1696      4  0.999573      viajes
65        2  0.117078       vuelo
65        3  0.882902       vuelo
276       3  0.999766      vuelos

[207 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 4, 3, 1])

## Building LDA Mallet Model

In [15]:
pip install --user gensim==3.8.3.

Note: you may need to restart the kernel to use updated packages.


In [16]:
pip install wget

Note: you may need to restart the kernel to use updated packages.


In [19]:
!python -m wget http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip


Saved under mallet-2.0.8.zip


In [22]:
! unzip ~/mallet-2.0.8.zip -d ~/

"unzip" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


In [18]:
# Download File: http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
mallet_path = 'C://Users//gabov//Desktop//aerolinea_sentiment//data//mallet-208//bin'
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)

CalledProcessError: Command 'C://Users//gabov//Desktop//aerolinea_sentiment//data//mallet-208//bin import-file --preserve-case --keep-sequence --remove-stopwords --token-regex "\S+" --input C:\Users\gabov\AppData\Local\Temp\7bdd9d_corpus.txt --output C:\Users\gabov\AppData\Local\Temp\7bdd9d_corpus.mallet' returned non-zero exit status 1.

In [ ]:
os.environ['MALLET_HOME'] = 'C:/Users/gabov/Desktop/aerolinea_sentiment/data/mallet-208'
mallet_path = '\data\mallet-208\bin\mallet' # you should NOT need to change this 
corpus_path = '\data' # you need to change this path to the directory containing your corpus of .txt files

In [ ]:
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=6, id2word=id2word)